### В этом коде мы используем два основных подхода: content-based filtering и collaborative filtering. Content-based filtering рекомендует аниме, похожие на то, которое пользователь уже посмотрел, на основе жанров. Collaborative filtering использует оценки других пользователей для предсказания того, что может понравиться конкретному пользователю. Гибридная система комбинирует оба этих подхода для более точных рекомендаций.

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import numpy as np

In [3]:
# Загрузка данных
anime = pd.read_csv('anime.csv', nrows=10000)
ratings = pd.read_csv('rating.csv', nrows=10000)

# Объединение данных аниме и рейтингов по столбцу anime_id
data = pd.merge(ratings, anime.drop('rating', axis=1), on='anime_id')
# Удаляем дубликаты
data.drop_duplicates(inplace=True)

data.head(10)

,user_id,anime_id,rating,name,genre,type,episodes,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
1,3,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
2,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
3,6,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
4,10,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
5,21,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
6,28,20,9,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
7,34,20,9,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
8,38,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
9,39,20,10,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9935 entries, 0 to 9934
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   9935 non-null   int64 
 1   anime_id  9935 non-null   int64 
 2   rating    9935 non-null   int64 
 3   name      9935 non-null   object
 4   genre     9935 non-null   object
 5   type      9935 non-null   object
 6   episodes  9935 non-null   object
 7   members   9935 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 621.1+ KB


In [23]:
# Построение модели на основе содержания (Content-Based Filtering)
# Используем TfidfVectorizer для преобразования жанров в числовые векторы
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['genre'].fillna(''))  # Обработка NaN значений
# Вычисляем косинусное сходство между аниме
cosine_sim = cosine_similarity(tfidf_matrix, dense_output=False)

In [24]:
def content_based_recommendations(title, df=data):
    # Функция для получения рекомендаций на основе названия аниме
    if title not in df['name'].values:
        return pd.Series([], dtype=object)  # Если аниме не найдено, возвращаем пустой список
    idx = df.index[df['name'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx].toarray().flatten()))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    anime_indices = [i[0] for i in sim_scores]
    return df['name'].iloc[anime_indices].head(10)

In [25]:
# Collaborative Filtering
# Создаем сводную таблицу рейтингов для каждого пользователя и аниме
pivot_table = data.pivot_table(index='user_id', columns='anime_id', values='rating', aggfunc='mean').fillna(0)
pivot_sparse = csr_matrix(pivot_table.values)
# Применяем SVD (разложение матрицы) для извлечения скрытых факторов
U, sigma, Vt = svds(pivot_sparse, k=50)  # Увеличиваем k для лучшей точности


In [26]:
def collaborative_filtering(user_id, df=data, pivot=pivot_table):
    # Функция для коллаборативной фильтрации
    if user_id not in pivot.index:
        return pd.Series([], dtype=object)  # Если пользователь не найден, возвращаем пустой список
    user_index = pivot.index.tolist().index(user_id)
    user_ratings = U[user_index]
    predicted_ratings = user_ratings.dot(np.diag(sigma)).dot(Vt)
    recommended_anime_ids = np.argsort(-predicted_ratings)[:10]
    recommended_anime = df.loc[df['anime_id'].isin(recommended_anime_ids), 'name'].unique()
    return pd.Series(recommended_anime)

In [27]:
# Гибридная система
def hybrid_recommendation(user_id, title):
    # Функция для гибридных рекомендаций, объединяющая оба подхода
    content_based = content_based_recommendations(title)
    collaborative = collaborative_filtering(user_id)
    
    # Объединяем результаты и удаляем дубликаты
    hybrid = pd.concat([content_based, collaborative]).drop_duplicates().head(10)
    return hybrid

In [29]:
# Тестирование системы
user_id_example = 1
title_example = 'Naruto'
print("Рекомендации:")
print(hybrid_recommendation(user_id_example, title_example))

Рекомендации:
0                              Naruto
0     School Rumble Ichi Gakki Hoshuu
1    Mermaid Melody Pichi Pichi Pitch
2                Angel&#039;s Feather
dtype: object
